**third rank**

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("ThirdHighestCountry").getOrCreate()
df = spark.read.format("delta").load("dbfs:/user/hive/warehouse/sales_country")
agg_df = df.groupBy("country").agg(sum("sales").alias("tot_sales"))
wnd = Window.orderBy(col("tot_sales").desc())
ranked_df = agg_df.withColumn("rank", dense_rank().over(wnd))
result = ranked_df.filter(col("rank") == 3)
result.show()


+-------+------------------+----+
|country|         tot_sales|rank|
+-------+------------------+----+
| France|1110916.5199999993|   3|
+-------+------------------+----+



**duplicate count**

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("ThirdHighestCountry").getOrCreate()
df = spark.read.format("delta").load("dbfs:/user/hive/warehouse/sales_country")

wnd = Window.partitionBy("country").orderBy("sales")
df = df.withColumn("rnk",row_number().over(wnd))
res = df.filter(col("rnk") > 1).count()
print(f"Number of rows with rank > 1 per country: {res}")


Number of rows with rank > 1 per country: 2804
